In [19]:
import numpy as np

def matprint(mat, fmt="g"):
    col_maxes = [max([len(("{:"+fmt+"}").format(x)) for x in col]) for col in mat.T]
    for x in mat:
        for i, y in enumerate(x):
            print(("{:"+str(col_maxes[i])+fmt+"}").format(y), end="  ")
        print("")

In [20]:
A = np.array(
    [
        [0.0, 3.7, 2.6, 0.0, 0.0],
        [8.9, 0.0, 0.0, 1.2, 0.0],
        [0.0, 0.0, 0.0, 1.9, 2.3],
        [0.0, 0.0, 0.0, 0.0, 0.0],
        [4.4, 0.0, 0.0, 2.7, 4.4]
    ]
)

k = 2
n = np.size(A, 1)

In [21]:
np.linalg.matrix_power(A, 2)

array([[32.93,  0.  ,  0.  ,  9.38,  5.98],
       [ 0.  , 32.93, 23.14,  0.  ,  0.  ],
       [10.12,  0.  ,  0.  ,  6.21, 10.12],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [19.36, 16.28, 11.44, 11.88, 19.36]])

In [22]:
matrix_pwr_sum = np.zeros((5,5))
for ii in range(n):
    matrix_pwr_sum += np.linalg.matrix_power(A, ii)
matprint(matrix_pwr_sum)

 1260.4  222.895  156.629  405.452  344.986  
536.154  1118.31   785.14  228.381  287.399  
583.823  202.198  143.085  250.585  313.389  
      0        0        0        1        0  
1381.94  939.193  659.974   590.76  742.611  


In [13]:
n

5